In [1]:
import tensorflow_hub as hub
import tensorflow as tf

import array

import pandas as pd
import pandas_profiling as pp
import numpy as np
from  sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from scipy import spatial


In [2]:
data_path = '/Users/amitjain/personal/machine-learning/Machine_Learning/competitions/google_quest/data/google-quest-challenge/train.csv' 

In [3]:
data = pd.read_csv(data_path)

In [5]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

OSError: SavedModel file does not exist at: /var/folders/w6/2r_6wc0j2sdg5m6dqgchnsnr0000gn/T/tfhub_modules/063d866c06683311b44b4992fd46003be952409c/{saved_model.pbtxt|saved_model.pb}

In [6]:
titles_embedding = embed(data.question_title.values)
question_body_embedding = embed(data.question_body.values)
answers_embedding = embed(data.answer.values)

In [7]:
print('Shape of title embedding', titles_embedding.shape)
print('Shape of question bofy embedding', question_body_embedding.shape)
print('Shape of answer embedding', answers_embedding.shape)

Shape of title embedding (6079, 512)
Shape of question bofy embedding (6079, 512)
Shape of answer embedding (6079, 512)


In [25]:
def similarity(vector_1, vector_2):
    result = [None] * vector_1.shape[0]
    for i in range(0, vector_1.shape[0]):
        result[i] = spatial.distance.cosine(vector_1[i], vector_2[2])
        
    return result

In [26]:
%%time
title_body_similarity = similarity(titles_embedding, question_body_embedding)
body_answer_similarity = similarity(question_body_embedding, answers_embedding)
title_answer_similarity = similarity(titles_embedding, answers_embedding)

CPU times: user 23.9 s, sys: 173 ms, total: 24.1 s
Wall time: 24.2 s


In [27]:
labels = ['question_asker_intent_understanding',
       'question_body_critical', 'question_conversational',
       'question_expect_short_answer', 'question_fact_seeking',
       'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfaction', 'answer_type_instructions',
       'answer_type_procedure', 'answer_type_reason_explanation',
       'answer_well_written']

In [29]:
data['title_body_similarity'] = title_body_similarity
data['body_answer_similaroty'] = body_answer_similarity
data['title_answer_similarity'] = title_answer_similarity

In [30]:
feature_cols = ['title_body_similarity', 'body_answer_similaroty', 'title_answer_similarity']

In [31]:
data[feature_cols].head()

,title_body_similarity,body_answer_similaroty,title_answer_similarity
0,0.791494,0.766909,0.914999
1,1.030054,0.790771,0.946088
2,0.798457,0.421404,0.859977
3,0.895016,0.790325,0.929174
4,0.917324,0.865642,0.943121


In [32]:
pp.ProfileReport(data[np.concatenate([labels, feature_cols])])

/opt/anaconda3/envs/Pytthon3/lib/python3.7/site-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,33
Number of observations,6079
Total Missing (%),0.0%
Total size in memory,1.5 MiB
Average record size in memory,264.0 B
Numeric,33
Categorical,0
Boolean,0
Date,0
Text (Unique),0
Rejected,0
